In [ ]:
''' IMPORTS '''

import math
from datetime import datetime

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import nfl_data_py as nfl

from resources.plotly_theme import nfl_template
from resources.get_nfl_data import get_matchups, get_team_info
from resources.epa_model import PicksModel, EPAUtility

pio.templates['nfl_template'] = nfl_template

In [ ]:
''' Retrain Model '''

picks_model = PicksModel()
picks_model.retrain_model()


In [ ]:
''' Team info '''

# Get team info
team_data = get_team_info()

In [ ]:
''' Predict Hypothetical Matchups '''

# Constants
PREDICTION_SEASON = 2025
PREDICTION_WEEK = 18

EXPORT = False

FOLDER = f'../visuals/week {PREDICTION_WEEK}/predictions'

# Matchups to predict
matchups = pd.DataFrame(columns=['home_team', 'away_team'])
matchups['home_team'] = ['PHI']
matchups['away_team'] = ['SF']

# Predict
picks_model = PicksModel()
predictions_df = picks_model.predict_matchups(matchups=matchups, moneyline_value=False)

# Colors and logos
predictions_df = predictions_df.merge(team_data.reset_index()[['team', 'team_color', 'team_logo_espn']], left_on='home_team', right_on='team', how='left').drop(columns=['team']).rename(columns={'team_color': 'home_color', 'team_logo_espn': 'home_logo'})
predictions_df = predictions_df.merge(team_data.reset_index()[['team', 'team_color', 'team_logo_espn']], left_on='away_team', right_on='team', how='left').drop(columns=['team']).rename(columns={'team_color': 'away_color', 'team_logo_espn': 'away_logo'})

print(predictions_df.to_string())


In [ ]:
''' Predict a Week '''

# Constants
PREDICTION_SEASON = 2025
PREDICTION_WEEK = 19

EXPORT = False

FOLDER = f'../visuals/week {PREDICTION_WEEK}/predictions'

# Get matchups
matchups = get_matchups(years=[PREDICTION_SEASON], include_postseason=True)
matchups = matchups.loc[matchups['week'] == PREDICTION_WEEK,:]

# Predict
picks_model = PicksModel()
predictions_df = picks_model.predict_matchups(matchups=matchups, moneyline_value=True)

# Colors and logos
predictions_df = predictions_df.merge(team_data.reset_index()[['team', 'team_color', 'team_logo_espn']], left_on='home_team', right_on='team', how='left').drop(columns=['team']).rename(columns={'team_color': 'home_color', 'team_logo_espn': 'home_logo'})
predictions_df = predictions_df.merge(team_data.reset_index()[['team', 'team_color', 'team_logo_espn']], left_on='away_team', right_on='team', how='left').drop(columns=['team']).rename(columns={'team_color': 'away_color', 'team_logo_espn': 'away_logo'})

print(predictions_df.to_string())

In [ ]:
print(predictions_df[['home_team', 'Home_Team_Last_16_EPA_ST_Play']].sort_values(by='Home_Team_Last_16_EPA_ST_Play', ascending=False))
print(predictions_df[['away_team', 'Away_Team_Last_16_EPA_ST_Play']].sort_values(by='Away_Team_Last_16_EPA_ST_Play', ascending=False))

## Visualize

In [ ]:
''' Win Probability / Picks - Pie Charts '''

# Create pie chart for each game
titles = []
pie_charts = []
winner_logos = []
for i in predictions_df.index:

    away_prob, home_prob = predictions_df.loc[i, ['prob_away', 'prob_home']]
    away_team, home_team = predictions_df.loc[i, ['away_team', 'home_team']]
    
    away_color, home_color = predictions_df.loc[i, ['away_color', 'home_color']]
    away_logo, home_logo = predictions_df.loc[i, ['away_logo', 'home_logo']]

    off_black = 'rgba(0,0,0,0.7)'
    pie_chart = go.Pie(
        values=[home_prob, away_prob],
        labels=[home_team, away_team],
        marker=dict(
            colors=[home_color if home_prob > away_prob else off_black, away_color if away_prob > home_prob else off_black], 
            pattern=dict(
                shape=["" if home_prob > away_prob else "x", "" if away_prob > home_prob else "x"],
                size=4,
            ),
            line=dict(color='#f3f3f3', width=2),
        ),
        textposition='outside',
        textinfo='percent+label',
        hole=0.6,
        textfont=dict(weight='bold'),
        sort=False
    )

    pie_charts.append(pie_chart)
    titles.append(f'{away_team} vs. {home_team}')
    winner_logos.append(away_logo if away_prob > home_prob else home_logo)


## Create Figure ##
N_COLS = 4 if len(predictions_df) >= 4 else len(predictions_df)
N_ROWS = math.ceil(len(predictions_df) / N_COLS)
domain = [[{"type": "domain"} for i in range(N_COLS)] for i in range(N_ROWS)]

H_SPACING = 0.075
V_SPACING = 0.075
picks_fig = make_subplots(rows=N_ROWS, cols=N_COLS, specs=domain, subplot_titles=titles, 
                    vertical_spacing=V_SPACING, horizontal_spacing=H_SPACING)

# Add charts to plot
i = 0

total_v_spacing = ((N_ROWS - 1)*V_SPACING)
total_v_available = 1 - total_v_spacing
row_size = total_v_available / N_ROWS

total_h_spacing = ((N_COLS - 1)*H_SPACING)
total_h_available = 1 - total_h_spacing
col_size = total_h_available / N_COLS

for r in range(N_ROWS):
    for c in range(N_COLS):
        # Pie
        picks_fig.add_trace(
            pie_charts[i],
            row=r+1,
            col=c+1
        )

        # Winner logo
        picks_fig.add_layout_image(
            source=winner_logos[i],  # The loaded image
            xref="paper",    # Reference x-coordinates to the x-axis
            yref="paper",    # Reference y-coordinates to the y-axis
            x=(col_size / 2) + (col_size*c + H_SPACING*c), # Y-coordinate of the image's center
            y=(1 - (row_size / 2)) - (row_size*r + V_SPACING*r), # Y-coordinate of the image's center
            sizex=.05,   # Width of the image in data units
            sizey=.05,   # Height of the image in data units
            xanchor="center", # Anchor the image by its center horizontally
            yanchor="middle", # Anchor the image by its middle vertically
            layer="above", # Place image above other plot elements
            opacity=0.9,
        )

        i += 1
        if i > len(predictions_df.index) - 1:
            break

# Format
picks_fig.for_each_annotation(lambda a: a.update(font=dict(size=14, weight='bold')))

picks_fig.update_layout(
    template='nfl_template',
    paper_bgcolor='#f0f0f0',
    title=dict(
        text=f'<B>NFL Week {PREDICTION_WEEK} <span style="color: #D5A15D">Picks</span></b><br><sup>Win Probability Model</sup>',
    ),
    margin=dict(t=100, b=50, l=50, r=50),
    showlegend=False,
    height=900,
    width=800
)

# Credits
picks_fig.add_annotation(
    text=f'EPA / Play from teams\' last 4, 8, 12, and 16 games, in all 3 phases<br>Figure & Model: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.05, 
    x=1,
    align='right'
)

picks_fig.show()

In [ ]:
''' Predictions '''

from PIL import Image
import requests
from io import BytesIO


BORDER_COLOR = '#989898'

tables = []
winner_logos = []
for i in predictions_df.index:
    
    away_team, home_team = predictions_df.loc[i, ['away_team', 'home_team']]
    away_prob, home_prob = predictions_df.loc[i, ['prob_away', 'prob_home']]
    away_team_ml, home_team_ml = predictions_df.loc[i, ['pred_away_ml_viz', 'pred_home_ml_viz']]
    away_color, home_color = predictions_df.loc[i, ['away_color', 'home_color']]
    away_logo, home_logo = predictions_df.loc[i, ['away_logo', 'home_logo']]
   
    # Spread
    spread_line = predictions_df.loc[i, 'spread_line']
    home_spread_odds = predictions_df.loc[i, 'home_spread_odds'].astype(int)
    away_spread_odds = predictions_df.loc[i, 'away_spread_odds'].astype(int)
    spread_pick = predictions_df.loc[i, 'spread_pick']

    away_spread = f'+{abs(spread_line)}' if spread_line > 0 else f'-{abs(spread_line)}'
    away_spread += f' ({away_spread_odds})'
    home_spread = f'-{abs(spread_line)}' if spread_line > 0 else f'+{abs(spread_line)}'
    home_spread += f' ({home_spread_odds})'

    # Total
    total_line = predictions_df.loc[i, 'total_line']
    total_pick = predictions_df.loc[i, 'total_pick']

    # ML
    home_moneyline = predictions_df.loc[i, 'home_moneyline'].astype(int)
    home_moneyline = f'+{abs(home_moneyline)}' if home_moneyline > 0 else f'-{abs(home_moneyline)}'
    away_moneyline = predictions_df.loc[i, 'away_moneyline'].astype(int)
    away_moneyline = f'+{abs(away_moneyline)}' if away_moneyline > 0 else f'-{abs(away_moneyline)}'
    moneyline_value = predictions_df.loc[i, 'moneyline_value']

    # Table
    table = go.Table(
        columnwidth=[2,4,3,2],
        header=dict(
            values=['', 'Spread', 'Total', 'ML'],
            line_color=['rgba(0,0,0,0)']+[BORDER_COLOR]*3,
            fill_color=['rgba(0,0,0,0)']+['white']*3,
            align=['center', 'center'],
            font=dict(size=10)
        ),
        cells=dict(
            values=[[away_team, home_team], 
                    [away_spread, home_spread], 
                    [f'O {total_line}', f'U {total_line}'],
                    [away_moneyline, home_moneyline]],
            line_color=[BORDER_COLOR]*4,
            line_width=1,
            fill_color=[[away_color, home_color],
                        ['white' if spread_pick == 'home' else 'rgba(75,181,67,0.7)', 'white' if spread_pick == 'away' else 'rgba(75,181,67,0.7)'],
                        ['white' if total_pick == 'over' else 'rgba(75,181,67,0.7)', 'white' if total_pick == 'under' else 'rgba(75,181,67,0.7)'],
                        ['white' if moneyline_value == 'home' else 'rgba(75,181,67,0.7)', 'white' if moneyline_value == 'away' else 'rgba(75,181,67,0.7)']],
            font=dict(
                color=[['white', 'white'],
                       ['black' if spread_pick == 'home' else 'white', 'black' if spread_pick == 'away' else 'white'],
                       ['black' if total_pick == 'over' else 'white', 'black' if total_pick == 'under' else 'white'],
                       ['black' if moneyline_value == 'home' else 'white', 'black' if moneyline_value == 'away' else 'white']]
            )
        )
    )
    tables.append(table)
    winner_logos.append(away_logo if away_prob > home_prob else home_logo)


## Make Figure
N_ROWS = math.ceil(len(tables) / 2)
N_COLS = 2
betting_sheet_fig = make_subplots(rows=N_ROWS, cols=N_COLS*2, 
                    specs=[[{"type": "table"}, {"type": "xy"}]*N_COLS for i in range(N_ROWS)],
                    subplot_titles=['', 'Projected<br>Winner']*len(tables),
                    horizontal_spacing=0.05,
                    column_widths=[6,1,6,1])

# Add tables / winner logos
i = 0
for r in range(1,N_ROWS+1):
    for c in range(1,N_COLS+1):

        ## Logo
        response = requests.get(winner_logos[i])
        img = Image.open(BytesIO(response.content))

        logo_trace = px.imshow(img=img)
        betting_sheet_fig.add_trace(
            logo_trace.data[0], 
            row=r, 
            col=c*2
        )

        ## Table
        betting_sheet_fig.add_trace(
            tables[i],
            row=r,
            col=(c*2)-1
        )

        i += 1
        if i > len(tables) - 1:
            break

    
## Formatting
betting_sheet_fig.update_annotations(font=dict(size=10, weight='bold'))

betting_sheet_fig.update_layout(
    template='nfl_template',
    paper_bgcolor='#f0f0f0',
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(
        text=f'<b>NFL Week {PREDICTION_WEEK} <span style="color: #D5A15D">Betting Sheet</span></b><br><sup>Scoring Model</sup>',
        y=0.965
    ),
    margin=dict(b=50, l=25, r=25),
    height=1200,#1000,
    width=700
)
betting_sheet_fig.update_xaxes(
    visible=False,
)
betting_sheet_fig.update_yaxes(
    visible=False,
)

# Credits
betting_sheet_fig.add_annotation(
    text=f'EPA / Play from teams\' last 4, 8, 12, and 16 games, in all 3 phases<br>Figure & Model: @clankeranalytic | Data: nfl_data_py | {datetime.today():%Y-%m-%d}',
    font=dict(size=8),
    showarrow=False,
    xref='paper',
    yref='paper',
    y=-0.025, 
    x=1,
    align='right'
)

betting_sheet_fig.show()

In [ ]:
''' Export '''

if EXPORT:
    predictions_df.to_excel(f'{FOLDER}/Week {PREDICTION_WEEK} Predictions.xlsx', index=False)
    
    pio.write_image(picks_fig, f'{FOLDER}/Week {PREDICTION_WEEK} Picks.png', scale=6, width=900, height=900)
    pio.write_image(betting_sheet_fig, f'{FOLDER}/Week {PREDICTION_WEEK} Betting Sheet.png', scale=6, width=700, height=1200)


# Results

In [ ]:

# Get week results
matchups = get_matchups(years=[PREDICTION_SEASON])
matchups = matchups.loc[matchups['week'] == PREDICTION_WEEK, ['game_id', 'home_score', 'away_score', 'winner', 'result', 'total']]

# Get model predictions
pred_week_viz_df = pd.read_excel(f'{FOLDER}/Week {PREDICTION_WEEK} Predictions.xlsx', sheet_name='Sheet1')
pred_week_viz_df = pred_week_viz_df.drop(columns=['home_score', 'away_score'])

## Get Results
week_results_df = pred_week_viz_df.merge(matchups, on='game_id', how='left')

## Evaluate

# Game pick
week_results_df['winner'] = np.where(week_results_df['winner'] == 0, week_results_df['away_team'], week_results_df['home_team'])
week_results_df['winner_correct?'] = np.where(week_results_df['winner'] == week_results_df['pred'], 1, 0)

# Spread picks
week_results_df['spread_result'] = np.where(week_results_df['result'] > week_results_df['spread_line'], 'home', 'away')
week_results_df['spread_correct?'] = np.where(week_results_df['spread_pick'] == week_results_df['spread_result'], 1, 0)

# Total picks
week_results_df['total_result'] = np.where(week_results_df['total'] > week_results_df['total_line'], 'over', 'under')
week_results_df['total_correct?'] = np.where(week_results_df['total_pick'] == week_results_df['total_result'], 1, 0)


# ## Add Actual EPA
# cols = ['Plays_O', 'EPA_O_Play', 'Plays_D','EPA_D_Play', 'Plays_ST', 'EPA_ST_Play']

# week_epa = master_epa_df.loc[master_epa_df.index.get_level_values(0) == c_master_week, :].reset_index()
# # print(week_epa.head().to_string())

# # Home Team EPA
# week_results_df = week_results_df.merge(week_epa[['team'] + cols], left_on='home_team', right_on='team', how='inner').rename(columns={
#     col: f'home_team_{col}' for col in cols
# }).drop(columns=['team'])

# # Away Team EPA
# week_results_df = week_results_df.merge(week_epa[['team'] + cols], left_on='away_team', right_on='team', how='inner').rename(columns={
#     col: f'away_team_{col}' for col in cols
# }).drop(columns=['team'])

week_results_df = week_results_df.reindex(columns=[
        'season', 'week', 'home_team', 'away_team', 'home_score', 'away_score',
        'prob_home', 'pred_home_ml_viz', 'prob_away', 'pred_away_ml_viz',
        'pred', 'winner', 'winner_correct?', 
        'pred_home_score', 'pred_away_score', 
        'spread_line', 'pred_spread', 'spread_pick', 'result', 'spread_result', 'spread_correct?',
        'total_line', 'pred_total', 'total_pick', 'total', 'total_result', 'total_correct?',
        'home_team_Plays_O', 'home_team_EPA_O_Play', 'home_team_EPA_D_Play', 'home_team_EPA_ST_Play', 
        'away_team_Plays_O', 'away_team_EPA_O_Play', 'away_team_EPA_D_Play', 'away_team_EPA_ST_Play'])

print(week_results_df.columns)
print(week_results_df.to_string())

## Export
# week_results_df.to_excel(f'{FOLDER}/Week {PREDICTION_WEEK} Results.xlsx', index=False, sheet_name='Results')